In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D,
                                   concatenate, BatchNormalization, Activation,
                                   GlobalAveragePooling2D, Dense, Flatten, Reshape)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall, MeanIoU, AUC
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from datetime import datetime
import json
import seaborn as sns
from sklearn.metrics import (precision_score, recall_score, f1_score,
                           jaccard_score, confusion_matrix, classification_report,
                           roc_curve, auc, RocCurveDisplay)
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
PREPROCESSED_DATA_PATH = '/content/drive/MyDrive/preprocessed_dataset'

def load_preprocessed_data(data_dir, split_name):
    """
    Load preprocessed images and masks from directory
    """
    images_dir = os.path.join(data_dir, split_name, 'images')
    masks_dir = os.path.join(data_dir, split_name, 'masks')

    if not os.path.exists(images_dir) or not os.path.exists(masks_dir):
        raise ValueError(f"Preprocessed data not found for {split_name}")

    # Get sorted lists of files
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png')])
    mask_files = sorted([f for f in os.listdir(masks_dir) if f.endswith('.png')])

    images = []
    masks = []

    for img_file, mask_file in tqdm(zip(image_files, mask_files), desc=f"Loading {split_name} data", total=len(image_files)):
        # Load image
        img_path = os.path.join(images_dir, img_file)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image.astype(np.float32) / 255.0  # Normalize

        # Load mask
        mask_path = os.path.join(masks_dir, mask_file)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = mask.astype(np.float32) / 255.0  # Normalize to 0-1

        # Add channel dimension to mask
        mask = np.expand_dims(mask, axis=-1)

        images.append(image)
        masks.append(mask)

    return np.array(images), np.array(masks)

def prepare_classification_data(images, masks, threshold=0.01):
    """
    Prepare classification data from segmentation data
    Returns images and labels (1 if oil spill present, 0 otherwise)
    """
    # Calculate the percentage of oil pixels in each mask
    oil_percentages = np.array([np.sum(mask) / (mask.size) for mask in masks])

    # Create binary labels: 1 if oil percentage > threshold, else 0
    labels = (oil_percentages > threshold).astype(np.int32)

    return images, labels

def calculate_class_weights_binary(y_train):
    """
    Calculate class weights for binary classification
    """
    class_counts = np.bincount(y_train)
    total = np.sum(class_counts)

    # Handle case where one class might be missing
    if len(class_counts) == 1:
        if class_counts[0] > 0:
            class_counts = np.array([class_counts[0], 0])
        else:
            class_counts = np.array([0, class_counts[0]])

    # Calculate weights (inverse of frequency)
    class_weights = {
        0: float(total / (2 * class_counts[0])) if class_counts[0] > 0 else 1.0,
        1: float(total / (2 * class_counts[1])) if class_counts[1] > 0 else 1.0
    }

    print(f"Class counts: No Oil Spill={class_counts[0]}, Oil Spill={class_counts[1]}")
    print(f"Class weights: {class_weights}")
    return class_weights

class UNetFeatureExtractor:
    def __init__(self, input_shape=(256, 256, 3)):
        self.input_shape = input_shape
        self.model = None
        self.feature_extractor = None

    def build_unet(self, filters=64, dropout_rate=0.5, batch_norm=True):
        """
        Build U-Net model for feature extraction
        """
        inputs = Input(self.input_shape)

        # Encoder (Contracting Path)
        def conv_block(x, filters, dropout_rate=0.0, batch_norm=True):
            x = Conv2D(filters, 3, padding='same')(x)
            if batch_norm:
                x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(filters, 3, padding='same')(x)
            if batch_norm:
                x = BatchNormalization()(x)
            x = Activation('relu')(x)

            if dropout_rate > 0:
                x = Dropout(dropout_rate)(x)
            return x

        # Encoder
        c1 = conv_block(inputs, filters, 0.0, batch_norm)
        p1 = MaxPooling2D((2, 2))(c1)

        c2 = conv_block(p1, filters*2, 0.0, batch_norm)
        p2 = MaxPooling2D((2, 2))(c2)

        c3 = conv_block(p2, filters*4, 0.0, batch_norm)
        p3 = MaxPooling2D((2, 2))(c3)

        c4 = conv_block(p3, filters*8, dropout_rate, batch_norm)
        p4 = MaxPooling2D((2, 2))(c4)

        # Bottleneck
        c5 = conv_block(p4, filters*16, dropout_rate, batch_norm)

        # Create model
        model = Model(inputs=inputs, outputs=c5)
        self.model = model
        return model

    def extract_features_batch(self, images, batch_size=32):
        """
        Extract features using U-Net encoder with batch processing
        """
        if self.model is None:
            raise ValueError("Model not built yet. Please build the model first.")

        features = []
        num_batches = int(np.ceil(len(images) / batch_size))

        for i in tqdm(range(num_batches), desc="Extracting features"):
            batch_start = i * batch_size
            batch_end = min((i + 1) * batch_size, len(images))
            batch_images = images[batch_start:batch_end]

            batch_features = self.model.predict(batch_images, verbose=0)
            features.append(batch_features)

        return np.vstack(features)

class OilSpillClassifier:
    def __init__(self, input_shape=(16, 16, 1024)):  # Default shape from U-Net bottleneck
        self.input_shape = input_shape
        self.model = None
        self.history = None
        self.metrics = {}

    def build_classifier(self):
        """
        Build classifier on top of U-Net features
        """
        inputs = Input(self.input_shape)

        # Global average pooling
        x = GlobalAveragePooling2D()(inputs)
        x = Dropout(0.5)(x)

        # Additional dense layers
        x = Dense(512, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.4)(x)

        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)

        x = Dense(128, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)

        # Output layer
        outputs = Dense(1, activation='sigmoid')(x)

        # Create model
        model = Model(inputs=inputs, outputs=outputs)
        self.model = model
        return model

    def compile_model(self, learning_rate=1e-4):
        """
        Compile the classification model
        """
        if self.model is None:
            raise ValueError("Model not built yet. Please build the model first.")

        self.model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='binary_crossentropy',
            metrics=['accuracy', Precision(), Recall(), AUC()]
        )

    def train(self, X_train, y_train, X_val, y_val, epochs=50, batch_size=32,
              callbacks=None, class_weight=None):
        """
        Train the classification model
        """
        if self.model is None:
            raise ValueError("Model not built yet. Please build the model first.")

        # Default callbacks
        if callbacks is None:
            callbacks = [
                ModelCheckpoint(
                    '/content/best_classifier_model.h5',
                    monitor='val_accuracy',
                    save_best_only=True,
                    mode='max',
                    verbose=1
                ),
                EarlyStopping(
                    monitor='val_accuracy',
                    patience=10,
                    restore_best_weights=True,
                    verbose=1
                ),
                ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.2,
                    patience=5,
                    min_lr=1e-7,
                    verbose=1
                )
            ]

        # Train the model
        self.history = self.model.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            class_weight=class_weight,
            verbose=1
        )

        return self.history

    def evaluate(self, X_test, y_test, threshold=0.5):
        """
        Evaluate the classification model
        """
        if self.model is None:
            raise ValueError("Model not trained yet. Please train the model first.")

        # Keras evaluation
        results = self.model.evaluate(X_test, y_test, verbose=0)
        metrics = {
            'loss': results[0],
            'accuracy': results[1],
            'precision': results[2],
            'recall': results[3],
            'auc': results[4]
        }

        # Additional metrics using sklearn
        y_pred_probs = self.model.predict(X_test, verbose=0)
        y_pred = (y_pred_probs > threshold).astype(int)

        metrics['f1_score'] = f1_score(y_test, y_pred)

        # Confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        metrics['confusion_matrix'] = cm.tolist()
        if cm.size == 4:
            metrics['tn'], metrics['fp'], metrics['fn'], metrics['tp'] = cm.ravel()
        else:
            metrics['tn'], metrics['fp'], metrics['fn'], metrics['tp'] = 0, 0, 0, 0

        # Classification report
        metrics['classification_report'] = classification_report(y_test, y_pred, output_dict=True)

        # ROC curve data
        fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
        metrics['roc_curve'] = {'fpr': fpr.tolist(), 'tpr': tpr.tolist()}
        metrics['roc_auc'] = auc(fpr, tpr)

        self.metrics = metrics
        return metrics

    def predict(self, features, threshold=0.5):
        """
        Predict class probabilities for features
        """
        if self.model is None:
            raise ValueError("Model not trained yet. Please train the model first.")

        # Predict
        predictions = self.model.predict(features, verbose=0)
        binary_predictions = (predictions > threshold).astype(int)

        return predictions.squeeze(), binary_predictions.squeeze()

    def plot_training_history(self, save_path=None):
        """
        Plot training history
        """
        if self.history is None:
            raise ValueError("No training history available. Please train the model first.")

        fig, axes = plt.subplots(2, 3, figsize=(18, 12))

        # Plot loss
        axes[0, 0].plot(self.history.history['loss'], label='Training Loss')
        axes[0, 0].plot(self.history.history['val_loss'], label='Validation Loss')
        axes[0, 0].set_title('Model Loss')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].legend()
        axes[0, 0].grid(True)

        # Plot accuracy
        axes[0, 1].plot(self.history.history['accuracy'], label='Training Accuracy')
        axes[0, 1].plot(self.history.history['val_accuracy'], label='Validation Accuracy')
        axes[0, 1].set_title('Model Accuracy')
        axes[0, 1].set_ylabel('Accuracy')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].legend()
        axes[0, 1].grid(True)

        # Plot precision
        axes[0, 2].plot(self.history.history['precision'], label='Training Precision')
        axes[0, 2].plot(self.history.history['val_precision'], label='Validation Precision')
        axes[0, 2].set_title('Model Precision')
        axes[0, 2].set_ylabel('Precision')
        axes[0, 2].set_xlabel('Epoch')
        axes[0, 2].legend()
        axes[0, 2].grid(True)

        # Plot recall
        axes[1, 0].plot(self.history.history['recall'], label='Training Recall')
        axes[1, 0].plot(self.history.history['val_recall'], label='Validation Recall')
        axes[1, 0].set_title('Model Recall')
        axes[1, 0].set_ylabel('Recall')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].legend()
        axes[1, 0].grid(True)

        # Plot AUC
        axes[1, 1].plot(self.history.history['auc'], label='Training AUC')
        axes[1, 1].plot(self.history.history['val_auc'], label='Validation AUC')
        axes[1, 1].set_title('Model AUC')
        axes[1, 1].set_ylabel('AUC')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].legend()
        axes[1, 1].grid(True)

        # Remove empty subplot
        axes[1, 2].set_visible(False)

        plt.tight_layout()

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()

    def plot_confusion_matrix(self, save_path=None):
        """
        Plot confusion matrix
        """
        if not self.metrics or 'confusion_matrix' not in self.metrics:
            raise ValueError("No evaluation metrics available. Please evaluate the model first.")

        cm = np.array(self.metrics['confusion_matrix'])
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['No Oil Spill', 'Oil Spill'],
                   yticklabels=['No Oil Spill', 'Oil Spill'])
        plt.title('Confusion Matrix - Classification')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()

    def plot_roc_curve(self, save_path=None):
        """
        Plot ROC curve
        """
        if not self.metrics or 'roc_curve' not in self.metrics:
            raise ValueError("No ROC curve data available. Please evaluate the model first.")

        fpr = self.metrics['roc_curve']['fpr']
        tpr = self.metrics['roc_curve']['tpr']
        roc_auc = self.metrics['roc_auc']

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random classifier')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()

    def save_model(self, model_path='/content/oil_spill_classifier.h5'):
        """
        Save the trained model
        """
        if self.model is None:
            raise ValueError("Model not trained yet. Please train the model first.")

        self.model.save(model_path)
        print(f"Classifier model saved to {model_path}")

    def load_model(self, model_path='/content/oil_spill_classifier.h5'):
        """
        Load a trained model
        """
        self.model = tf.keras.models.load_model(model_path)
        print(f"Classifier model loaded from {model_path}")

    def generate_report(self, save_path='/content/classification_report.json'):
        """
        Generate a comprehensive classification report
        """
        report = {
            'timestamp': datetime.now().isoformat(),
            'model_architecture': 'U-Net Feature-based Classifier',
            'input_shape': self.input_shape,
            'metrics': self.metrics,
            'training_history': {
                'final_training_loss': self.history.history['loss'][-1] if self.history else None,
                'final_validation_loss': self.history.history['val_loss'][-1] if self.history else None,
                'final_training_accuracy': self.history.history['accuracy'][-1] if self.history else None,
                'final_validation_accuracy': self.history.history['val_accuracy'][-1] if self.history else None,
                'epochs_trained': len(self.history.history['loss']) if self.history else 0
            }
        }

        with open(save_path, 'w') as f:
            json.dump(report, f, indent=4)

        print(f"Classification report saved to {save_path}")
        return report

def train_classification_pipeline():
    """
    Complete pipeline for training oil spill classification using U-Net features
    """
    # Load preprocessed data
    data_dir = PREPROCESSED_DATA_PATH

    print("Loading data...")
    X_train_seg, y_train_seg = load_preprocessed_data(data_dir, 'train')
    X_val_seg, y_val_seg = load_preprocessed_data(data_dir, 'val')
    X_test_seg, y_test_seg = load_preprocessed_data(data_dir, 'test')

    print(f"Segmentation data shapes:")
    print(f"Train: {X_train_seg.shape}, {y_train_seg.shape}")
    print(f"Val: {X_val_seg.shape}, {y_val_seg.shape}")
    print(f"Test: {X_test_seg.shape}, {y_test_seg.shape}")

    # Prepare classification data
    print("\nPreparing classification data...")
    X_train_cls, y_train_cls = prepare_classification_data(X_train_seg, y_train_seg)
    X_val_cls, y_val_cls = prepare_classification_data(X_val_seg, y_val_seg)
    X_test_cls, y_test_cls = prepare_classification_data(X_test_seg, y_test_seg)

    print(f"Classification data shapes:")
    print(f"Train: {X_train_cls.shape}, {y_train_cls.shape}")
    print(f"Val: {X_val_cls.shape}, {y_val_cls.shape}")
    print(f"Test: {X_test_cls.shape}, {y_test_cls.shape}")

    # Calculate class weights
    cls_class_weights = calculate_class_weights_binary(y_train_cls)

    # Build U-Net feature extractor
    print("\nBuilding U-Net feature extractor...")
    feature_extractor = UNetFeatureExtractor(input_shape=(256, 256, 3))
    unet_model = feature_extractor.build_unet(filters=64, dropout_rate=0.3, batch_norm=True)

    # Extract features with batch processing
    print("\nExtracting features from training data...")
    X_train_features = feature_extractor.extract_features_batch(X_train_cls, batch_size=32)
    X_val_features = feature_extractor.extract_features_batch(X_val_cls, batch_size=32)
    X_test_features = feature_extractor.extract_features_batch(X_test_cls, batch_size=32)

    print(f"Feature shapes:")
    print(f"Train features: {X_train_features.shape}")
    print(f"Val features: {X_val_features.shape}")
    print(f"Test features: {X_test_features.shape}")

    # Build and train classifier
    print("\n" + "="*50)
    print("TRAINING CLASSIFICATION MODEL")
    print("="*50)

    classifier = OilSpillClassifier(input_shape=X_train_features.shape[1:])
    cls_model = classifier.build_classifier()
    cls_model.summary()

    classifier.compile_model(learning_rate=1e-4)

    cls_history = classifier.train(
        X_train_features, y_train_cls, X_val_features, y_val_cls,
        epochs=30,
        batch_size=32,
        class_weight=cls_class_weights
    )

    # Evaluate the model
    cls_metrics = classifier.evaluate(X_test_features, y_test_cls)
    print("\nClassification Evaluation Metrics:")
    for metric, value in cls_metrics.items():
        if metric not in ['confusion_matrix', 'classification_report', 'roc_curve']:
            print(f"{metric}: {value:.4f}")

    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_test_cls,
                               (classifier.model.predict(X_test_features) > 0.5).astype(int)))

    # Generate reports and visualizations
    cls_report = classifier.generate_report('/content/classification_report.json')

    # Save models
    classifier.save_model('/content/oil_spill_classifier.h5')

    # Plot results
    classifier.plot_training_history('/content/classification_training_history.png')
    classifier.plot_confusion_matrix('/content/classification_confusion_matrix.png')
    classifier.plot_roc_curve('/content/roc_curve.png')

    print("\n" + "="*50)
    print("CLASSIFICATION TRAINING COMPLETE!")
    print("="*50)

    return classifier, cls_metrics, feature_extractor

# Alternative: Simple CNN classifier (much faster)
def train_simple_cnn_classifier():
    """
    Train a simple CNN classifier for faster results
    """
    # Load preprocessed data
    data_dir = PREPROCESSED_DATA_PATH

    print("Loading data...")
    X_train_seg, y_train_seg = load_preprocessed_data(data_dir, 'train')
    X_val_seg, y_val_seg = load_preprocessed_data(data_dir, 'val')
    X_test_seg, y_test_seg = load_preprocessed_data(data_dir, 'test')

    # Prepare classification data
    X_train_cls, y_train_cls = prepare_classification_data(X_train_seg, y_train_seg)
    X_val_cls, y_val_cls = prepare_classification_data(X_val_seg, y_val_seg)
    X_test_cls, y_test_cls = prepare_classification_data(X_test_seg, y_test_seg)

    # Calculate class weights
    cls_class_weights = calculate_class_weights_binary(y_train_cls)

    # Build simple CNN model
    print("\nBuilding simple CNN classifier...")
    model = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['accuracy', Precision(), Recall(), AUC()]
    )

    model.summary()

    # Train the model
    print("\nTraining simple CNN classifier...")
    history = model.fit(
        X_train_cls, y_train_cls,
        batch_size=32,
        epochs=20,
        validation_data=(X_val_cls, y_val_cls),
        class_weight=cls_class_weights,
        verbose=1
    )

    # Evaluate
    results = model.evaluate(X_test_cls, y_test_cls, verbose=0)
    print(f"\nTest accuracy: {results[1]:.4f}")
    print(f"Test precision: {results[2]:.4f}")
    print(f"Test recall: {results[3]:.4f}")

    # Save model
    model.save('/content/simple_cnn_classifier.h5')
    print("Simple CNN classifier saved!")

    return model, history

# Run the classification pipeline
if __name__ == "__main__":
    print("Choose classification approach:")
    print("1. U-Net Feature-based Classifier (More accurate but slower)")
    print("2. Simple CNN Classifier (Faster but less accurate)")

    choice = input("Enter your choice (1 or 2): ")

    if choice == "1":
        # Train U-Net feature-based classifier
        classifier, cls_metrics, feature_extractor = train_classification_pipeline()
    elif choice == "2":
        # Train simple CNN classifier
        model, history = train_simple_cnn_classifier()
    else:
        print("Invalid choice. Running U-Net feature-based classifier...")
        classifier, cls_metrics, feature_extractor = train_classification_pipeline()

Mounted at /content/drive
Choose classification approach:
1. U-Net Feature-based Classifier (More accurate but slower)
2. Simple CNN Classifier (Faster but less accurate)
Loading data...


Loading test data: 100%|██████████| 254/254 [00:53<00:00,  4.76it/s]


Class counts: No Oil Spill=370, Oil Spill=1296
Class weights: {0: 2.2513513513513512, 1: 0.6427469135802469}

Building simple CNN classifier...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

...

 Total params: 26,210,625 (99.99 MB)

 Trainable params: 26,210,625 (99.99 MB)

 Non-trainable params: 0 (0.00 B)


Training simple CNN classifier...
Epoch 1/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 216s 4s/step - accuracy: 0.5750 - auc: 0.5890 - loss: 0.6869 - precision: 0.8152 - recall: 0.5744 - val_accuracy: 0.8670 - val_auc: 0.9128 - val_loss: 0.3762 - val_precision: 0.9096 - val_recall: 0.9264
Epoch 2/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 257s 4s/step - accuracy: 0.7484 - auc: 0.8136 - loss: 0.5314 - precision: 0.9111 - recall: 0.7492 - val_accuracy: 0.6355 - val_auc: 0.9266 - val_loss: 0.5976 - val_precision: 0.9890 - val_recall: 0.5521
Epoch 3/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 215s 4s/step - accuracy: 0.7398 - auc: 0.8620 - loss: 0.4592 - precision: 0.9486 - recall: 0.7076 - val_accuracy: 0.7143 - val_auc: 0.9426 - val_loss: 0.5024 - val_precision: 0.9907 - val_recall: 0.6503
Epoch 4/20
53/53 ━━━━━━━━━━━━━━━━━━━━ 209s 4s/step - accuracy: 0.7913 - auc: 0.9088 - loss: 0.3948 - precision: 0.9490 - recall: 0.7738 - val_accuracy: 0.9458 - val_auc: 0.9810 - val_loss: 0.1638 - val_precision: 0.9419 - val_recall: 0.9939
E


Test accuracy: 0.9724
Test precision: 0.9949
Test recall: 0.9700
Simple CNN classifier saved!
